<a href="https://colab.research.google.com/github/canerskrc/Borusan_Kod_Adi_Dijital/blob/main/Borusan_day8_advanced_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("car details v4.csv")

In [8]:
df["Engine_cc"]= df["Engine"].astype(str).str.extract(r"(\d+(?:\.\d+)?)").astype(float)
mp = df["Max Power"].astype(str).str.extract(r"(?P<bhp>\d+(?:\.\d+)?)\s*bhp\s*@\s*(?P<power_rpm>\d+)")

df["Max_Power_bhp"] = pd.to_numeric(mp["bhp"],errors="coerce") # coerce = sayıya dönüşemeyen bir değer olursa bunu NaN şeklinde ifade edecek.
df["Max_Power_rpm"] = pd.to_numeric(mp["power_rpm"], errors="coerce") # 87 beygir -> 87.0 / 6000rpm -> 6000.0 şeklinde yazılacak.

#max_Torque Sütunu içinden Nm ve rpm değerlerini çıkarma

mt = df["Max Torque"].astype(str).str.extract(r"(?P<Nm>\d+(?:\.\d+)?)\s*Nm\s*@\s*(?P<torque_rpm>\d+)")

df["Max_Touque_Nm"] = pd.to_numeric(mt["Nm"],errors="coerce") # coerce = sayıya dönüşemeyen bir değer olursa bunu NaN şeklinde ifade edecek.
df["Max_Torque_rpm"] = pd.to_numeric(mt["torque_rpm"], errors="coerce")

##Araç yaşı ve yıllık km

In [9]:
CURRENT_YEAR = 2025
df["Car_Age"] = (CURRENT_YEAR - df["Year"]).clip(lower=0)
df["Km_per_year"] = df["Kilometer"] / df["Car_Age"].replace(0,1) #replace(0,1) = araç yaşı kısmında 0 çıkan değerler olursa bu değerleri 1 ile değiştirecek ki bölme hatalarını ortadan kaldırsın.



##Bellek optimizasyonu

In [11]:
mem_before =  df.memory_usage(deep=True).sum() / 1024**2
print(f"Önceki bellek kullanımı: {mem_before:.2f} MB")

Önceki bellek kullanımı: 1.60 MB


In [12]:
df_opt = df.copy()
obj_cols = df_opt.select_dtypes(include="object").columns
low_card = [c for c in obj_cols if df_opt[c].nunique(dropna=False) / len(df_opt) < 0.5]

for c in low_card:
  df_opt[c] = df_opt[c].astype("category")

#sayısal sütunları kontrol ederek downcast yapma

num_cols = df_opt.select_dtypes(include=["int64","float64"]).columns
for c in num_cols:
  if pd.api.types.is_integer_dtype(df_opt[c]):
      df_opt[c] = pd.to_numeric(df_opt[c], downcast="integer")
  else:
    df_opt[c] = pd.to_numeric(df_opt[c], downcast="float")

mem_after = df_opt.memory_usage(deep=True).sum() / 1024**2
print(f"Sonraki bellek kullanımı: {mem_after:.2f} MB")

#tasarruf yüzdesi
saving =100 * (1 - mem_after / mem_before)
print(f"Tasarruf oranı: %{saving:.1f}")

Sonraki bellek kullanımı: 0.36 MB
Tasarruf oranı: %77.8


In [16]:
resent_premium = df.query("Year >= 2015 and Kilometer <= 160000 and Price >= @df['Price'].quantile(0.5)")

ValueError: data type must provide an itemsize

In [19]:
#make bazında MAD işlemi

def mad(series):
  return np.median(np.abs(series - np.median(series.dropna())))

mad_by_make = df.groupby("Make")["Price"].apply(mad).rename("Price_MAD_in_Make") # her marka için tek bir mad değeri çıkacak.

df = df.merge(mad_by_make, left_on="Make", right_index=True, how="left")
df.head(15)

,Make,Model,Price,Year,Kilometer,Fuel Type,Transmission,Location,Color,Owner,...,Engine_cc,Max_Power_bhp,Max_Power_rpm,Max_Touque_Nm,Max_Torque_rpm,Car_Age,Km_per_year,Price_MAD_in_Make_x,Price_MAD_in_Make_y,Price_MAD_in_Make
0,Honda,Amaze 1.2 VX i-VTEC,505000,2017,87150,Petrol,Manual,Pune,Grey,First,...,1198.0,87.0,6000.0,109.0000,4500.0,8,10893.750000,195000.0,195000.0,195000.0
1,Maruti Suzuki,Swift DZire VDI,450000,2014,75000,Diesel,Manual,Ludhiana,White,Second,...,1248.0,74.0,4000.0,190.0000,2000.0,11,6818.181818,170000.0,170000.0,170000.0
2,Hyundai,i10 Magna 1.2 Kappa2,220000,2011,67000,Petrol,Manual,Lucknow,Maroon,First,...,1197.0,79.0,6000.0,112.7619,4000.0,14,4785.714286,245000.0,245000.0,245000.0
3,Toyota,Glanza G,799000,2019,37500,Petrol,Manual,Mangalore,Red,First,...,1197.0,82.0,6000.0,113.0000,4200.0,6,6250.000000,727000.0,727000.0,727000.0
4,Toyota,Innova 2.4 VX 7 STR [2016-2020],1950000,2018,69000,Diesel,Manual,Mumbai,Grey,First,...,2393.0,148.0,3400.0,343.0000,1400.0,7,9857.142857,727000.0,727000.0,727000.0
5,Maruti Suzuki,Ciaz ZXi,675000,2017,73315,Petrol,Manual,Pune,Grey,First,...,1373.0,91.0,6000.0,130.0000,4000.0,8,9164.375000,170000.0,170000.0,170000.0
6,Mercedes-Benz,CLA 200 Petrol Sport,1898999,2015,47000,Petrol,Automatic,Mumbai,White,Second,...,1991.0,181.0,5500.0,300.0000,1200.0,10,4700.000000,1975000.0,1975000.0,1975000.0
7,BMW,X1 xDrive20d M Sport,2650000,2017,75000,Diesel,Automatic,Coimbatore,White,Second,...,1995.0,188.0,4000.0,400.0000,1750.0,8,9375.000000,1250000.0,1250000.0,1250000.0
8,Skoda,Octavia 1.8 TSI Style Plus AT [2017],1390000,2017,56000,Petrol,Automatic,Mumbai,White,First,...,1798.0,177.0,5100.0,250.0000,1250.0,8,7000.000000,575000.0,575000.0,575000.0
9,Nissan,Terrano XL (D),575000,2015,85000,Diesel,Manual,Mumbai,White,First,...,1461.0,84.0,3750.0,200.0000,1900.0,10,8500.000000,97500.0,97500.0,97500.0
